# Синхронизация с гит

In [6]:
# --- БЛОК №1: НАЧАЛО РАБОТЫ (выполнять в каждой новой сессии) ---

from google.colab import drive, userdata
import os

# 1. Монтируем Google Drive
drive.mount('/content/drive')

# 2. Получаем секретный токен
GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')

# 3. Указываем ваше имя и email (как на GitHub)
GIT_USERNAME = "kirichich1" # <-- Ваше имя пользователя
GIT_EMAIL = "kirichich@bk.ru" # <-- Ваша почта

# 4. !!! ПРЕДСТАВЛЯЕМСЯ СИСТЕМЕ GIT !!!
# Эта конфигурация будет действовать до конца текущей сессии
!git config --global user.name "{GIT_USERNAME}"
!git config --global user.email "{GIT_EMAIL}"

# 5. Переходим в директорию проекта
PROJECT_PATH = "/content/drive/MyDrive/GitHub/no2_prediction_pipeline"
%cd {PROJECT_PATH}

# 6. (Рекомендуется) Скачиваем последние изменения с GitHub на случай, если вы работали с другого устройства
!git pull

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/GitHub/no2_prediction_pipeline
Already up to date.


# Импорт нужных инструментов


In [1]:
import ee
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
ee.Authenticate()
ee.Initialize(project='pollutionmlproject')

In [3]:
!pip install earthengine-api

# Выгрузка данных

In [8]:
# --- ПАРАМЕТРЫ ГОРОДА ---
CITY_GEOM = ee.Geometry.Point([52.2978, 54.9014]).buffer(25000)
CITY_NAME = 'Almetyevsk'

# ==============================================================================
# == УСТАНОВИТЕ ГОД ДЛЯ ВЫГРУЗКИ ===============================================
# ==============================================================================
YEAR_TO_DOWNLOAD = 2019 # <--- МЕНЯЙТЕ ГОД ЗДЕСЬ
# ==============================================================================

# --- Параметры фильтрации S5P ---
PIXEL_CLOUD_THRESHOLD = 0.5
SOLAR_ZENITH_ANGLE_THRESHOLD = 80.0

# ==============================================================================
# == БЛОК 2: ФУНКЦИИ-СБОРЩИКИ ДАННЫХ (S5P обновлена для AAI) ===================
# ==============================================================================

def get_s5p_data_for_date(target_date, geometry):
    """
    Получает данные NO2 и AAI за одну дату.
    КРИТИЧЕСКИ ВАЖНО: Маска облачности берется из NO2 и применяется к AAI.
    """
    try:
        start = ee.Date(target_date)
        end = start.advance(1, 'day')

        # 1. Запрашиваем коллекцию NO2 (источник маски облачности)
        s5p_no2_coll = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_NO2') \
            .filterBounds(geometry).filterDate(start, end)

        # 2. Запрашиваем коллекцию AAI (данные, которые нужно отфильтровать)
        s5p_aai_coll = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_AER_AI') \
            .filterBounds(geometry).filterDate(start, end)

        # Если нет снимка NO2, мы не можем ни получить NO2, ни сделать маску для AAI.
        if s5p_no2_coll.size().getInfo() == 0:
            return None

        # Берем средние снимки за день
        image_no2 = s5p_no2_coll.mean()

        # Проверяем, есть ли AAI. Если нет, создаем пустую переменную, чтобы код не упал
        has_aai = s5p_aai_coll.size().getInfo() > 0
        image_aai = s5p_aai_coll.mean() if has_aai else ee.Image.constant(None)

        # --- СОЗДАНИЕ МАСКИ (по данным NO2) ---
        cloud_mask = image_no2.select('cloud_fraction').lt(PIXEL_CLOUD_THRESHOLD)
        sza_mask = image_no2.select('solar_zenith_angle').lt(SOLAR_ZENITH_ANGLE_THRESHOLD)
        combined_mask = cloud_mask.And(sza_mask)

        # --- ПРИМЕНЕНИЕ МАСКИ К ОБОИМ ПРОДУКТАМ ---
        # Фильтруем NO2
        no2_filtered = image_no2.updateMask(combined_mask)

        # Фильтруем AAI той же самой маской! (Реализация вашего плана)
        aai_filtered = image_aai.updateMask(combined_mask) if has_aai else image_aai

        # Сбор статистики (Reduce Region)
        stats_no2 = no2_filtered.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=geometry,
            scale=1000,
            maxPixels=1e9
        ).getInfo()

        # Отдельно собираем облачность (без маски облачности, чтобы знать реальное покрытие)
        cloud_stats = image_no2.select('cloud_fraction').reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=geometry,
            scale=1000,
            maxPixels=1e9
        ).getInfo()

        stats_aai = {}
        if has_aai:
            stats_aai = aai_filtered.select('absorbing_aerosol_index').reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=geometry,
                scale=1000,
                maxPixels=1e9
            ).getInfo()

        return {
            'no2_trop_mean': stats_no2.get('tropospheric_NO2_column_number_density'),
            'cloud_frac_mean': cloud_stats.get('cloud_fraction'),
            'aai_mean': stats_aai.get('absorbing_aerosol_index') if has_aai else None
        }

    except Exception as e:
        print(f"Ошибка S5P на дате {target_date}: {e}")
        return None

def get_era5_data_for_date(target_date, geometry):
    """
    Функция ERA5 без изменений.
    """
    try:
        start = ee.Date(target_date)
        end = start.advance(1, 'day')
        era5_bands = ['temperature_2m', 'surface_pressure',
                      'u_component_of_wind_10m', 'v_component_of_wind_10m']

        image = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY') \
                  .filterBounds(geometry) \
                  .filterDate(start, end) \
                  .select(era5_bands) \
                  .mean()

        if image:
            data = image.reduceRegion(
                reducer=ee.Reducer.mean(), geometry=geometry, scale=11132, maxPixels=1e9
            ).getInfo()

            temp_celsius = data.get('temperature_2m') - 273.15 if data.get('temperature_2m') else None
            pressure_hpa = data.get('surface_pressure') / 100 if data.get('surface_pressure') else None

            return {
                'temperature_celsius': temp_celsius,
                'pressure_hpa': pressure_hpa,
                'u_wind_10m': data.get('u_component_of_wind_10m'),
                'v_wind_10m': data.get('v_component_of_wind_10m')
            }
        return None
    except Exception as e:
        print(f"Ошибка ERA5 на дате {target_date}: {e}"); return None

# ==============================================================================
# == БЛОК 3: ОСНОВНОЙ ЦИКЛ СБОРА ДАННЫХ =========================================
# ==============================================================================

all_data = []
print(f"Начинаю сбор данных (NO2 + AAI + ERA5) для города: {CITY_NAME} за {YEAR_TO_DOWNLOAD} год")

start_of_period = f"{YEAR_TO_DOWNLOAD}-01-01"
end_of_period = f"{YEAR_TO_DOWNLOAD}-12-31"

print(f"--- Период: с {start_of_period} по {end_of_period} ---")
date_range = pd.date_range(start=start_of_period, end=end_of_period, freq='D')

for date in tqdm(date_range, desc=f"Год {YEAR_TO_DOWNLOAD}"):
    current_date_str = date.strftime('%Y-%m-%d')

    # Теперь эта функция возвращает и NO2, и AAI
    s5p_data = get_s5p_data_for_date(current_date_str, CITY_GEOM)
    era5_data = get_era5_data_for_date(current_date_str, CITY_GEOM)

    row = {'date': date, 'city': CITY_NAME}

    if s5p_data:
        row.update(s5p_data)
    if era5_data:
        row.update(era5_data)

    all_data.append(row)

# ==============================================================================
# == БЛОК 4: ФОРМИРОВАНИЕ И СОХРАНЕНИЕ ИТОГОВОГО ФАЙЛА ========================
# ==============================================================================
final_df = pd.DataFrame(all_data)

# Обновленный порядок колонок, включая AAI
column_order = [
    'date', 'city',
    'no2_trop_mean', 'aai_mean', 'cloud_frac_mean', # Добавил aai_mean сюда
    'temperature_celsius', 'pressure_hpa', 'u_wind_10m', 'v_wind_10m'
]

# Пересортировка колонок (с защитой, если вдруг данных не было вообще и колонка не создалась)
existing_cols = [col for col in column_order if col in final_df.columns]
final_df = final_df[existing_cols]

# Заполнение пропусков AAI нулями (согласно вашему Плану Шаг 2)
if 'aai_mean' in final_df.columns:
    final_df['aai_mean'] = final_df['aai_mean'].fillna(0)
else:
    # Если за весь период не нашлось ни одного пикселя AAI, создаем колонку с нулями
    final_df['aai_mean'] = 0

drive.mount('/content/drive')

output_filename = f'{CITY_NAME}_data_{YEAR_TO_DOWNLOAD}_FULL_YEAR.csv'
final_df.to_csv(f'/content/drive/MyDrive/GitHub/no2_prediction_pipeline/data/raw/add_predictors_Alm/{output_filename}', index=False)

print(f"\nСбор данных завершен.")
print(f"Данные по AAI отфильтрованы по маске облачности NO2 (< {PIXEL_CLOUD_THRESHOLD}).")
print(f"Пропуски в AAI заполнены нулями.")
print(f"Файл сохранен: {output_filename}")

Начинаю сбор данных (NO2 + AAI + ERA5) для города: Almetyevsk за 2019 год
--- Период: с 2019-01-01 по 2019-12-31 ---


Год 2019:   0%|          | 0/365 [00:00<?, ?it/s]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Сбор данных завершен.
Данные по AAI отфильтрованы по маске облачности NO2 (< 0.5).
Пропуски в AAI заполнены нулями.
Файл сохранен: Almetyevsk_data_2019_FULL_YEAR.csv


In [9]:
final_df.head(50)

,date,city,no2_trop_mean,aai_mean,cloud_frac_mean,temperature_celsius,pressure_hpa,u_wind_10m,v_wind_10m
0,2019-01-01,Almetyevsk,NaN,0.000000,0.875685,-14.471187,1002.268750,0.949313,2.578113
1,2019-01-02,Almetyevsk,NaN,-0.996572,0.623044,-6.622771,1000.232828,0.652604,3.897729
2,2019-01-03,Almetyevsk,NaN,-1.908676,0.742129,-8.037156,1002.912985,0.086032,3.436004
3,2019-01-04,Almetyevsk,NaN,0.000000,0.964583,-10.152386,998.543529,-0.141057,3.752814
4,2019-01-05,Almetyevsk,NaN,0.000000,0.962941,-8.717924,988.175337,-1.200693,4.094333
5,2019-01-06,Almetyevsk,0.000046,-0.710835,0.411502,-8.631217,988.007903,2.784127,1.687065
6,2019-01-07,Almetyevsk,0.000024,-0.771160,0.444262,-15.605239,993.362533,2.127752,-0.309154
7,2019-01-08,Almetyevsk,NaN,-1.560769,0.498666,-15.354326,988.926989,2.989918,1.698387
8,2019-01-09,Almetyevsk,NaN,0.000000,0.878756,-14.073399,993.991583,2.270756,1.612903
9,2019-01-10,Almetyevsk,0.000033,-0.734887,0.348721,-11.428949,999.715530,2.197021,1.447529


In [10]:
final_df['no2_trop_mean'].isna().sum()

np.int64(159)

In [11]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   date                 365 non-null    datetime64[ns]
 1   city                 365 non-null    object        
 2   no2_trop_mean        206 non-null    float64       
 3   aai_mean             365 non-null    float64       
 4   cloud_frac_mean      354 non-null    float64       
 5   temperature_celsius  365 non-null    float64       
 6   pressure_hpa         365 non-null    float64       
 7   u_wind_10m           365 non-null    float64       
 8   v_wind_10m           365 non-null    float64       
dtypes: datetime64[ns](1), float64(7), object(1)
memory usage: 25.8+ KB


In [12]:
final_df['no2_trop_mean'].isna().sum()

np.int64(159)

# Коммит для гит

In [ ]:
# --- БЛОК №2: СОХРАНЕНИЕ РЕЗУЛЬТАТОВ (в конце работы) ---

# 1. Проверяем статус (опционально)
!git status

# 2. ДОБАВЛЯЕМ файлы в "коробку" для отправки. Это то, что вы пропустили.
!git add .

# 3. ПОДПИСЫВАЕМ "коробку" (делаем коммит)
!git commit -m "add Alm data-export script"

# 4. ОТПРАВЛЯЕМ "коробку" на GitHub
!git push